In [2]:
import json

In [ ]:
import json
from pathlib import Path

def combine_train_test_from_file(json_file_path):
    """Process a single JSON file and return its combined data."""
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    
    combined = [(item['input'], item['output']) for item in data['train'] + data['test']]
    
    return combined

def process_all_json_files(folder_path):
    """Process all JSON files in the given folder."""
    master_list = []
    
    # Get all JSON files in the folder
    json_files = list(Path(folder_path).glob('*.json'))
    
    # Process each file
    for json_file in json_files:
        file_data = combine_train_test_from_file(json_file)
        master_list.append(file_data)
        print(f"Processed {json_file.name}: {len(file_data)} examples")
    
    return master_list

# Example usage
train_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/training'
test_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/evaluation'
train_data = process_all_json_files(train_path)
test_data = process_all_json_files(test_path)

# save train_data and test_data to a file
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'w') as f:
    json.dump({'train': train_data, 'test': test_data}, f)

Processed 4cd1b7b2.json: 4 examples
Processed 9f41bd9c.json: 3 examples
Processed 7953d61e.json: 6 examples
Processed 9d9215db.json: 4 examples
Processed 40f6cd08.json: 4 examples
Processed 9968a131.json: 3 examples
Processed 4c5c2cf0.json: 4 examples
Processed 913fb3ed.json: 5 examples
Processed 6df30ad6.json: 6 examples
Processed 7e2bad24.json: 5 examples
Processed ea786f4a.json: 4 examples
Processed f18ec8cc.json: 6 examples
Processed 83302e8f.json: 4 examples
Processed 2faf500b.json: 4 examples
Processed 50aad11f.json: 4 examples
Processed c7d4e6ad.json: 3 examples
Processed a78176bb.json: 4 examples
Processed a5313dff.json: 4 examples
Processed afe3afe9.json: 4 examples
Processed 98c475bf.json: 5 examples
Processed ecdecbb3.json: 4 examples
Processed ddf7fa4f.json: 4 examples
Processed cfb2ce5a.json: 4 examples
Processed 522fdd07.json: 5 examples
Processed b9630600.json: 4 examples
Processed a8c38be5.json: 3 examples
Processed ae4f1146.json: 5 examples
Processed 94133066.json: 4 e

In [ ]:
# combined_data has the structure:
# {'train': [[(input1, output1),(input2, output2),...], # task 1
#            [(input1, output1),(input2, output2),...], # task 2
#               ...
#            [(input1, output1),(input2, output2),...]], # task n

#  'test': [[(input1, output1),(input2, output2),...], # task 1
#           [(input1, output1),(input2, output2),...], # task 2
#           ...
#           [(input1, output1),(input2, output2),...]], # task n
# }

In [4]:
from functions import *
# create fake examples like [(input1, output1),(input2, output2),...]
fake_examples = [([[1,2],[3,4]], [[1,2,3],[3,4,5]]), ([[2,3],[4,5]], [[6,1],[9,5]])]
# tokenize the fake examples
tokenized_examples = tokenize_arc_oneshot(fake_examples, False, 1000)
a,b,c = tokenized_examples

In [ ]:
for i in range(len(a)):
    print(i, a[i], b[i])